# Chapter 3: Introduction to Data Analysis in Python Polars 

## Inspecting a DataFrame

### How to do it...

In [1]:
import polars as pl

In [2]:
df = pl.read_csv('../data/covid_19_dealths.csv')

In [3]:
df.head(3)

Data As Of,Start Date,End Date,Group,Year,Month,State,Sex,Age Group,COVID-19 Deaths,Total Deaths,Pneumonia Deaths,Pneumonia and COVID-19 Deaths,Influenza Deaths,"Pneumonia, Influenza, or COVID-19 Deaths",Footnote
str,str,str,str,str,str,str,str,str,i64,i64,i64,i64,i64,i64,str
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,null,"""United States""","""All Sexes""","""All Ages""",1146774,12303399,1162844,569264,22229,1760095,null
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,null,"""United States""","""All Sexes""","""Under 1 year""",519,73213,1056,95,64,1541,null
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,null,"""United States""","""All Sexes""","""0-17 years""",1696,130970,2961,424,509,4716,null


In [4]:
df.tail(5)

Data As Of,Start Date,End Date,Group,Year,Month,State,Sex,Age Group,COVID-19 Deaths,Total Deaths,Pneumonia Deaths,Pneumonia and COVID-19 Deaths,Influenza Deaths,"Pneumonia, Influenza, or COVID-19 Deaths",Footnote
str,str,str,str,str,str,str,str,str,i64,i64,i64,i64,i64,i64,str
"""09/27/2023""","""09/01/2023""","""09/23/2023""","""By Month""","""2023""","""9""","""Puerto Rico""","""Female""","""50-64 years""",null,75,14,null,0,14,"""One or more da…"
"""09/27/2023""","""09/01/2023""","""09/23/2023""","""By Month""","""2023""","""9""","""Puerto Rico""","""Female""","""55-64 years""",0,65,10,0,0,10,null
"""09/27/2023""","""09/01/2023""","""09/23/2023""","""By Month""","""2023""","""9""","""Puerto Rico""","""Female""","""65-74 years""",null,91,null,null,0,null,"""One or more da…"
"""09/27/2023""","""09/01/2023""","""09/23/2023""","""By Month""","""2023""","""9""","""Puerto Rico""","""Female""","""75-84 years""",null,211,36,null,0,38,"""One or more da…"
"""09/27/2023""","""09/01/2023""","""09/23/2023""","""By Month""","""2023""","""9""","""Puerto Rico""","""Female""","""85 years and o…",null,265,42,null,null,44,"""One or more da…"


In [5]:
df.glimpse(max_items_per_column=3)

Rows: 137700
Columns: 16
$ Data As Of                               <str> '09/27/2023', '09/27/2023', '09/27/2023'
$ Start Date                               <str> '01/01/2020', '01/01/2020', '01/01/2020'
$ End Date                                 <str> '09/23/2023', '09/23/2023', '09/23/2023'
$ Group                                    <str> 'By Total', 'By Total', 'By Total'
$ Year                                     <str> None, None, None
$ Month                                    <str> None, None, None
$ State                                    <str> 'United States', 'United States', 'United States'
$ Sex                                      <str> 'All Sexes', 'All Sexes', 'All Sexes'
$ Age Group                                <str> 'All Ages', 'Under 1 year', '0-17 years'
$ COVID-19 Deaths                          <i64> 1146774, 519, 1696
$ Total Deaths                             <i64> 12303399, 73213, 130970
$ Pneumonia Deaths                         <i64> 1162844, 1056, 2961
$ P

In [6]:
df.estimated_size('mb')

37.42431926727295

In [7]:
import polars.selectors as cs
df.select(cs.numeric()).describe()

describe,COVID-19 Deaths,Total Deaths,Pneumonia Deaths,Pneumonia and COVID-19 Deaths,Influenza Deaths,"Pneumonia, Influenza, or COVID-19 Deaths"
str,f64,f64,f64,f64,f64,f64
"""count""",98270.0,118191.0,92836.0,100816.0,111012.0,93467.0
"""null_count""",39430.0,19509.0,44864.0,36884.0,26688.0,44233.0
"""mean""",313.586547,2841.952585,336.597085,152.513411,5.002468,505.491778
"""std""",5992.341375,56201.384331,6126.573599,2980.886938,110.606691,9256.951591
"""min""",0.0,0.0,0.0,0.0,0.0,0.0
"""25%""",0.0,43.0,0.0,0.0,0.0,0.0
"""50%""",0.0,153.0,18.0,0.0,0.0,25.0
"""75%""",50.0,657.0,74.0,21.0,0.0,107.0
"""max""",1.146774e6,1.2303399e7,1.162844e6,569264.0,22229.0,1.760095e6


In [8]:
df.null_count()

Data As Of,Start Date,End Date,Group,Year,Month,State,Sex,Age Group,COVID-19 Deaths,Total Deaths,Pneumonia Deaths,Pneumonia and COVID-19 Deaths,Influenza Deaths,"Pneumonia, Influenza, or COVID-19 Deaths",Footnote
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,2754,13770,0,0,0,39430,19509,44864,36884,26688,44233,39804


### There is more...

In [9]:
print(df.head())

shape: (5, 16)
┌────────────┬───────────┬───────────┬──────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ Data As Of ┆ Start     ┆ End Date  ┆ Group    ┆ … ┆ Pneumonia ┆ Influenza ┆ Pneumonia ┆ Footnote │
│ ---        ┆ Date      ┆ ---       ┆ ---      ┆   ┆ and       ┆ Deaths    ┆ , Influen ┆ ---      │
│ str        ┆ ---       ┆ str       ┆ str      ┆   ┆ COVID-19  ┆ ---       ┆ za, or    ┆ str      │
│            ┆ str       ┆           ┆          ┆   ┆ Deaths    ┆ i64       ┆ COVID-1…  ┆          │
│            ┆           ┆           ┆          ┆   ┆ ---       ┆           ┆ ---       ┆          │
│            ┆           ┆           ┆          ┆   ┆ i64       ┆           ┆ i64       ┆          │
╞════════════╪═══════════╪═══════════╪══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 09/27/2023 ┆ 01/01/202 ┆ 09/23/202 ┆ By Total ┆ … ┆ 569264    ┆ 22229     ┆ 1760095   ┆ null     │
│            ┆ 0         ┆ 3         ┆          ┆   ┆           ┆           

In [10]:
with pl.Config() as config:
    config.set_tbl_cols(11)
    print(df.head(2))

shape: (2, 16)
┌────────┬────────┬────────┬────────┬──────┬───────┬───┬────────┬────────┬────────┬────────┬───────┐
│ Data   ┆ Start  ┆ End    ┆ Group  ┆ Year ┆ Month ┆ … ┆ Pneumo ┆ Pneumo ┆ Influe ┆ Pneumo ┆ Footn │
│ As Of  ┆ Date   ┆ Date   ┆ ---    ┆ ---  ┆ ---   ┆   ┆ nia    ┆ nia    ┆ nza    ┆ nia,   ┆ ote   │
│ ---    ┆ ---    ┆ ---    ┆ str    ┆ str  ┆ str   ┆   ┆ Deaths ┆ and    ┆ Deaths ┆ Influe ┆ ---   │
│ str    ┆ str    ┆ str    ┆        ┆      ┆       ┆   ┆ ---    ┆ COVID- ┆ ---    ┆ nza,   ┆ str   │
│        ┆        ┆        ┆        ┆      ┆       ┆   ┆ i64    ┆ 19     ┆ i64    ┆ or COV ┆       │
│        ┆        ┆        ┆        ┆      ┆       ┆   ┆        ┆ Deaths ┆        ┆ ID-1…  ┆       │
│        ┆        ┆        ┆        ┆      ┆       ┆   ┆        ┆ ---    ┆        ┆ ---    ┆       │
│        ┆        ┆        ┆        ┆      ┆       ┆   ┆        ┆ i64    ┆        ┆ i64    ┆       │
╞════════╪════════╪════════╪════════╪══════╪═══════╪═══╪════════╪════════╪══

In [11]:
pl.Config.set_tbl_cols(11)
print(df.head(2))

shape: (2, 16)
┌────────┬────────┬────────┬────────┬──────┬───────┬───┬────────┬────────┬────────┬────────┬───────┐
│ Data   ┆ Start  ┆ End    ┆ Group  ┆ Year ┆ Month ┆ … ┆ Pneumo ┆ Pneumo ┆ Influe ┆ Pneumo ┆ Footn │
│ As Of  ┆ Date   ┆ Date   ┆ ---    ┆ ---  ┆ ---   ┆   ┆ nia    ┆ nia    ┆ nza    ┆ nia,   ┆ ote   │
│ ---    ┆ ---    ┆ ---    ┆ str    ┆ str  ┆ str   ┆   ┆ Deaths ┆ and    ┆ Deaths ┆ Influe ┆ ---   │
│ str    ┆ str    ┆ str    ┆        ┆      ┆       ┆   ┆ ---    ┆ COVID- ┆ ---    ┆ nza,   ┆ str   │
│        ┆        ┆        ┆        ┆      ┆       ┆   ┆ i64    ┆ 19     ┆ i64    ┆ or COV ┆       │
│        ┆        ┆        ┆        ┆      ┆       ┆   ┆        ┆ Deaths ┆        ┆ ID-1…  ┆       │
│        ┆        ┆        ┆        ┆      ┆       ┆   ┆        ┆ ---    ┆        ┆ ---    ┆       │
│        ┆        ┆        ┆        ┆      ┆       ┆   ┆        ┆ i64    ┆        ┆ i64    ┆       │
╞════════╪════════╪════════╪════════╪══════╪═══════╪═══╪════════╪════════╪══

## Casting data types

### How to do it...

In [12]:
import polars as pl

In [13]:
df = pl.read_csv('../data/covid_19_dealths.csv')
df.head()

Data As Of,Start Date,End Date,Group,Year,…,Pneumonia Deaths,Pneumonia and COVID-19 Deaths,Influenza Deaths,"Pneumonia, Influenza, or COVID-19 Deaths",Footnote
str,str,str,str,str,…,i64,i64,i64,i64,str
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,…,1162844,569264,22229,1760095,null
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,…,1056,95,64,1541,null
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,…,2961,424,509,4716,null
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,…,692,66,177,1079,null
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,…,818,143,219,1390,null


In [14]:
df.with_columns(
        pl.col('Data As Of').str.strptime(pl.Date, '%m/%d/%Y'),
        pl.col('Start Date').str.strptime(pl.Date, '%m/%d/%Y'),
        pl.col('End Date').str.strptime(pl.Date, '%m/%d/%Y'),
        pl.col('End Date').str.to_date('%m/%d/%Y').alias('End Date 2'),
        pl.col('Year').cast(pl.Int64)
).head()


Data As Of,Start Date,End Date,Group,Year,…,Pneumonia and COVID-19 Deaths,Influenza Deaths,"Pneumonia, Influenza, or COVID-19 Deaths",Footnote,End Date 2
date,date,date,str,i64,…,i64,i64,i64,str,date
2023-09-27,2020-01-01,2023-09-23,"""By Total""",null,…,569264,22229,1760095,null,2023-09-23
2023-09-27,2020-01-01,2023-09-23,"""By Total""",null,…,95,64,1541,null,2023-09-23
2023-09-27,2020-01-01,2023-09-23,"""By Total""",null,…,424,509,4716,null,2023-09-23
2023-09-27,2020-01-01,2023-09-23,"""By Total""",null,…,66,177,1079,null,2023-09-23
2023-09-27,2020-01-01,2023-09-23,"""By Total""",null,…,143,219,1390,null,2023-09-23


In [15]:
updated_df = (
    df.with_columns(
        pl.col('Data As Of').str.strptime(pl.Date, '%m/%d/%Y'),
        pl.col('Start Date').str.strptime(pl.Date, '%m/%d/%Y'),
        pl.col('End Date').str.strptime(pl.Date, '%m/%d/%Y'),
        pl.col('End Date').str.to_date('%m/%d/%Y').alias('End Date 2'),
        pl.col('Year').cast(pl.Int64)
    )
)

In [16]:
lf = pl.scan_csv('../data/covid_19_dealths.csv')
lf.with_columns(
        pl.col('Data As Of').str.strptime(pl.Date, '%m/%d/%Y'),
        pl.col('Start Date').str.strptime(pl.Date, '%m/%d/%Y'),
        pl.col('End Date').str.strptime(pl.Date, '%m/%d/%Y'),
        pl.col('End Date').str.to_date('%m/%d/%Y').alias('End Date 2'),
        pl.col('Year').cast(pl.Int64)
).collect().head()

Data As Of,Start Date,End Date,Group,Year,…,Pneumonia and COVID-19 Deaths,Influenza Deaths,"Pneumonia, Influenza, or COVID-19 Deaths",Footnote,End Date 2
date,date,date,str,i64,…,i64,i64,i64,str,date
2023-09-27,2020-01-01,2023-09-23,"""By Total""",null,…,569264,22229,1760095,null,2023-09-23
2023-09-27,2020-01-01,2023-09-23,"""By Total""",null,…,95,64,1541,null,2023-09-23
2023-09-27,2020-01-01,2023-09-23,"""By Total""",null,…,424,509,4716,null,2023-09-23
2023-09-27,2020-01-01,2023-09-23,"""By Total""",null,…,66,177,1079,null,2023-09-23
2023-09-27,2020-01-01,2023-09-23,"""By Total""",null,…,143,219,1390,null,2023-09-23


## Finding and removing duplicates values 

### How to do it

In [17]:
import polars as pl

In [18]:
df = pl.read_csv('../data/covid_19_dealths.csv')
df.head()

Data As Of,Start Date,End Date,Group,Year,…,Pneumonia Deaths,Pneumonia and COVID-19 Deaths,Influenza Deaths,"Pneumonia, Influenza, or COVID-19 Deaths",Footnote
str,str,str,str,str,…,i64,i64,i64,i64,str
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,…,1162844,569264,22229,1760095,null
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,…,1056,95,64,1541,null
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,…,2961,424,509,4716,null
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,…,692,66,177,1079,null
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,…,818,143,219,1390,null


In [19]:
df.shape

(137700, 16)

In [20]:
df.is_duplicated().sum()

0

In [21]:
df.is_unique().sum()

137700

In [22]:
df.n_unique()

137700

In [23]:
df.select(pl.all().n_unique())

Data As Of,Start Date,End Date,Group,Year,…,Pneumonia Deaths,Pneumonia and COVID-19 Deaths,Influenza Deaths,"Pneumonia, Influenza, or COVID-19 Deaths",Footnote
u32,u32,u32,u32,u32,…,u32,u32,u32,u32,u32
1,45,45,3,5,…,3556,2533,493,4264,2


In [24]:
df.n_unique(subset=['Start Date', 'End Date'])

50

In [25]:
(
    df
    .unique(subset=['Start Date', 'End Date'], keep='first')
    .select(row_count=pl.count())
)

row_count
u32
50


In [26]:
rows_to_keep = df.select(['Year', 'COVID-19 Deaths']).is_unique()
rows_to_keep.sum()

3940

In [27]:
df.filter(rows_to_keep).shape

(3940, 16)

In [28]:
df.filter(rows_to_keep).head()

Data As Of,Start Date,End Date,Group,Year,…,Pneumonia Deaths,Pneumonia and COVID-19 Deaths,Influenza Deaths,"Pneumonia, Influenza, or COVID-19 Deaths",Footnote
str,str,str,str,str,…,i64,i64,i64,i64,str
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,…,1162844,569264,22229,1760095,null
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,…,1056,95,64,1541,null
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,…,2961,424,509,4716,null
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,…,692,66,177,1079,null
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,…,818,143,219,1390,null


### There is more...

In [29]:
df.approx_n_unique()

Data As Of,Start Date,End Date,Group,Year,…,Pneumonia Deaths,Pneumonia and COVID-19 Deaths,Influenza Deaths,"Pneumonia, Influenza, or COVID-19 Deaths",Footnote
u32,u32,u32,u32,u32,…,u32,u32,u32,u32,u32
1,45,44,3,5,…,3587,2536,496,4263,2


## Masking sensitive data

### How to do it...

In [30]:
import polars as pl

In [31]:
df = pl.read_csv('../data/covid_19_dealths.csv')
df.head()

Data As Of,Start Date,End Date,Group,Year,…,Pneumonia Deaths,Pneumonia and COVID-19 Deaths,Influenza Deaths,"Pneumonia, Influenza, or COVID-19 Deaths",Footnote
str,str,str,str,str,…,i64,i64,i64,i64,str
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,…,1162844,569264,22229,1760095,null
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,…,1056,95,64,1541,null
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,…,2961,424,509,4716,null
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,…,692,66,177,1079,null
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,…,818,143,219,1390,null


In [32]:
import random

def get_random_nums(num_list, length):
    random_nums = ''.join(str(n) for n in random.sample(num_list, length))
    return random_nums

In [33]:
fake_ssns = []
nums = [n for n in range(0, 10)]

for i in range(0, df.height):
    part_1 = get_random_nums(nums, 3)
    part_2 = get_random_nums(nums, 2)
    part_3 = get_random_nums(nums, 4)
    fake_ssn = part_1 + '-' + part_2 + '-' + part_3
    fake_ssns.append(fake_ssn)

fake_ssns_df = pl.DataFrame({'SSN': fake_ssns})
fake_ssns_df.head()

SSN
str
"""078-07-3074"""
"""180-64-9315"""
"""149-56-0846"""
"""153-01-5036"""
"""976-26-7940"""


In [34]:
df = pl.concat([df, fake_ssns_df], how='horizontal')

In [35]:
df.select(
    ('XXX-XX-XX' + pl.col('SSN').str.slice(9, 2)).alias('SSN Masked')
).head()

SSN Masked
str
"""XXX-XX-XX74"""
"""XXX-XX-XX15"""
"""XXX-XX-XX46"""
"""XXX-XX-XX36"""
"""XXX-XX-XX40"""


In [36]:
df.select(
    ('XXX-XX-XX' + pl.col('SSN').str.slice(9, 2)).alias('SSN Masked'),
    
).head()

SSN Masked
str
"""XXX-XX-XX74"""
"""XXX-XX-XX15"""
"""XXX-XX-XX46"""
"""XXX-XX-XX36"""
"""XXX-XX-XX40"""


In [37]:
df.select(
    pl.col('SSN').hash()
).head()

SSN
u64
16688716547289931698
15739169009208351387
13236024879537598761
16141549672241121801
8396280402906047174


## Visualizing data using Plotly

### How to do it...

In [1]:
import polars as pl
import plotly.express as px

In [2]:
age_groups = ['0-17 years', '18-29 years', '30-39 years', '40-49 years', '50-64 years', '65-74 years', '75-84 years', '85 years and over', 'All Ages']

df = (
    pl.read_csv('../data/covid_19_dealths.csv')
    .filter(
        pl.col('Month').is_not_null(),
        pl.col('Age Group').is_in(age_groups),
    )
)
df.head()

Data As Of,Start Date,End Date,Group,Year,Month,State,Sex,Age Group,COVID-19 Deaths,Total Deaths,Pneumonia Deaths,Pneumonia and COVID-19 Deaths,Influenza Deaths,"Pneumonia, Influenza, or COVID-19 Deaths",Footnote
str,str,str,str,str,str,str,str,str,i64,i64,i64,i64,i64,i64,str
"""09/27/2023""","""01/01/2020""","""01/31/2020""","""By Month""","""2020""","""1""","""United States""","""All Sexes""","""All Ages""",6,264677,17909,3,2125,20037,null
"""09/27/2023""","""01/01/2020""","""01/31/2020""","""By Month""","""2020""","""1""","""United States""","""All Sexes""","""0-17 years""",0,2966,90,0,63,153,null
"""09/27/2023""","""01/01/2020""","""01/31/2020""","""By Month""","""2020""","""1""","""United States""","""All Sexes""","""18-29 years""",0,4426,114,0,54,168,null
"""09/27/2023""","""01/01/2020""","""01/31/2020""","""By Month""","""2020""","""1""","""United States""","""All Sexes""","""30-39 years""",0,6475,246,0,112,358,null
"""09/27/2023""","""01/01/2020""","""01/31/2020""","""By Month""","""2020""","""1""","""United States""","""All Sexes""","""40-49 years""",0,9792,485,0,151,636,null


In [3]:
df = (
    df.
    with_columns(
        pl.col('Data As Of').str.strptime(pl.Date, '%m/%d/%Y'),
        pl.col('Start Date').str.strptime(pl.Date, '%m/%d/%Y'),
        pl.col('End Date').str.strptime(pl.Date, '%m/%d/%Y'),
        pl.col('Year').cast(pl.Int64),
        pl.col('Month').cast(pl.Int64)
    )
) 
df.head()

Data As Of,Start Date,End Date,Group,Year,Month,State,Sex,Age Group,COVID-19 Deaths,Total Deaths,Pneumonia Deaths,Pneumonia and COVID-19 Deaths,Influenza Deaths,"Pneumonia, Influenza, or COVID-19 Deaths",Footnote
date,date,date,str,i64,i64,str,str,str,i64,i64,i64,i64,i64,i64,str
2023-09-27,2020-01-01,2020-01-31,"""By Month""",2020,1,"""United States""","""All Sexes""","""All Ages""",6,264677,17909,3,2125,20037,null
2023-09-27,2020-01-01,2020-01-31,"""By Month""",2020,1,"""United States""","""All Sexes""","""0-17 years""",0,2966,90,0,63,153,null
2023-09-27,2020-01-01,2020-01-31,"""By Month""",2020,1,"""United States""","""All Sexes""","""18-29 years""",0,4426,114,0,54,168,null
2023-09-27,2020-01-01,2020-01-31,"""By Month""",2020,1,"""United States""","""All Sexes""","""30-39 years""",0,6475,246,0,112,358,null
2023-09-27,2020-01-01,2020-01-31,"""By Month""",2020,1,"""United States""","""All Sexes""","""40-49 years""",0,9792,485,0,151,636,null


In [6]:
covid_deaths_by_age = (
    df
    .filter(
        pl.col('State')=='United States',
        pl.col('Year') == 2023,
        pl.col('Age Group') != 'All Ages',
        pl.col('Sex') == 'All Sexes'
    )
    .group_by('Age Group')
    .agg(pl.col('COVID-19 Deaths').sum())
    .sort(by='COVID-19 Deaths', descending=True)
)

fig = px.bar(
    covid_deaths_by_age, 
    x='Age Group', 
    y='COVID-19 Deaths', 
    title='COVID Deaths 2023 by Age Group - As of 9/27/23'
)

fig.update_layout(xaxis_title=None)
fig.show()


In [8]:
covid_deaths_by_top_5_states = (
    df
    .filter(
        pl.col('State') != 'United States',
        pl.col('Year') == 2023,
        pl.col('Age Group') == 'All Ages',
        pl.col('Sex') == 'All Sexes'
    )
    .group_by('State')
    .agg(pl.col('COVID-19 Deaths').sum())
    .sort(by='COVID-19 Deaths', descending=True)
    .head()
)

fig = px.bar(
    covid_deaths_by_top_5_states, 
    x='State', 
    y='COVID-19 Deaths', 
    title='COVID Deaths 2023 by Top 5 States - As of 9/27/23',
)

fig.update_layout(xaxis_title=None)
fig.show()

In [11]:
covid_deaths_by_sex = (
    df
    .filter(
        pl.col('State') == 'United States',
        pl.col('Year') == 2023,
        pl.col('Age Group') == 'All Ages',
        pl.col('Sex') != 'All Sexes'
    )
    .group_by('Sex')
    .agg(pl.col('COVID-19 Deaths').sum())
    .sort(by='COVID-19 Deaths', descending=True)
    .head()
)

fig = px.bar(
    covid_deaths_by_sex, 
    x='Sex', 
    y='COVID-19 Deaths', 
    title='COVID Deaths 2023 by Sex - As of 9/27/23',
    text_auto='.2s'
)

fig.update_layout(xaxis_title=None)
fig.update_traces(width = 0.3, textfont_size=12, textangle=0, textposition='inside')
fig.show()

In [12]:
from us_state_mappings import us_state_division_dict

covid_deaths_vs_flu_deaths = (
    df
    .with_columns(
        pl.col('State').replace(us_state_division_dict, default='Others').alias('Division')
    )
    .filter(
        pl.col('State') != 'United States',
        pl.col('Age Group') != 'All Ages',
        pl.col('Sex') != 'All Sexes',
        pl.col('Year') == 2023
    )
    .group_by('State', 'Division')
    .agg(
        pl.col('COVID-19 Deaths').sum(),
        pl.col('Influenza Deaths').sum(),
        pl.col('Pneumonia Deaths').sum()
    )
)

fig = px.scatter(
    covid_deaths_vs_flu_deaths, 
    x='COVID-19 Deaths', 
    y='Influenza Deaths', 
    color='Division',
    size='Pneumonia Deaths',
    hover_name='State',
    title='COVID-19, Influenza, and Pneumonia Deaths 2023 by US States and Divisions'
)

fig.show()

/Users/Yuki/Desktop/Polars-Cookbook/.venv/lib/python3.11/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [13]:
monthly_treand_by_year = (
    df
    .filter(
        pl.col('State') == 'United States',
        pl.col('Age Group') == 'All Ages',
        pl.col('Sex') == 'All Sexes'
    )
    .group_by('Year', 'Month')
    .agg(
        pl.col('COVID-19 Deaths').sum(),
    )
    .sort(by='Month')
)

fig = px.line(
    monthly_treand_by_year, 
    x='Month', 
    y='COVID-19 Deaths', 
    color='Year',
    title='COVID-19 Deaths Monthly Trend - United States',
    line_shape='spline'
)

fig.update_xaxes(dtick = 1)
fig.update_layout(legend_traceorder='reversed')
fig.show()

/Users/Yuki/Desktop/Polars-Cookbook/.venv/lib/python3.11/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



## Detecting and handling outliers  

### How to do it...

In [46]:
import polars as pl
import plotly 
df = pl.from_pandas(plotly.data.iris())
df.head()

sepal_length,sepal_width,petal_length,petal_width,species,species_id
f64,f64,f64,f64,str,i64
5.1,3.5,1.4,0.2,"""setosa""",1
4.9,3.0,1.4,0.2,"""setosa""",1
4.7,3.2,1.3,0.2,"""setosa""",1
4.6,3.1,1.5,0.2,"""setosa""",1
5.0,3.6,1.4,0.2,"""setosa""",1


In [47]:
import plotly.express as px

fig = px.box(df, y='sepal_width', width=500)
fig.show()

In [48]:
q1 = pl.col('sepal_width').quantile(0.25)
q3 = pl.col('sepal_width').quantile(0.75)
iqr = q3 - q1
threshold = 1.5
lower_limit = q1 - iqr * threshold
upper_limit = q3 + iqr * threshold

df.filter(
    (pl.col('sepal_width') < lower_limit) | (pl.col('sepal_width') > upper_limit)
).head()

sepal_length,sepal_width,petal_length,petal_width,species,species_id
f64,f64,f64,f64,str,i64
5.7,4.4,1.5,0.4,"""setosa""",1
5.2,4.1,1.5,0.1,"""setosa""",1
5.5,4.2,1.4,0.2,"""setosa""",1
5.0,2.0,3.5,1.0,"""versicolor""",2


In [49]:
is_outlier_iqr = (pl.col('sepal_width') < lower_limit) | (pl.col('sepal_width') > upper_limit)
df_iqr_outlier_removed = (
    df
    .filter(is_outlier_iqr == False)
)
df_iqr_outlier_removed.filter(is_outlier_iqr)

sepal_length,sepal_width,petal_length,petal_width,species,species_id
f64,f64,f64,f64,str,i64


In [50]:
df_iqr_outlier_replaced = (
    df
    .with_columns(
        pl.when(is_outlier_iqr == True)
        .then(pl.col('sepal_width').median())
        .otherwise(pl.col('sepal_width'))
        .alias('sepal_width')
    )
)
df_iqr_outlier_replaced.filter(is_outlier_iqr)

sepal_length,sepal_width,petal_length,petal_width,species,species_id
f64,f64,f64,f64,str,i64


In [51]:
df_zscore = (
    df.with_columns(
       sepal_width_zscore=(pl.col('sepal_width') - pl.col('sepal_width').mean()) / pl.col('sepal_width').std()
    )
)
df_zscore.head()

sepal_length,sepal_width,petal_length,petal_width,species,species_id,sepal_width_zscore
f64,f64,f64,f64,str,i64,f64
5.1,3.5,1.4,0.2,"""setosa""",1,1.028611
4.9,3.0,1.4,0.2,"""setosa""",1,-0.12454
4.7,3.2,1.3,0.2,"""setosa""",1,0.33672
4.6,3.1,1.5,0.2,"""setosa""",1,0.10609
5.0,3.6,1.4,0.2,"""setosa""",1,1.259242


In [52]:
is_outlier_z_score = (pl.col('sepal_width_zscore') > 3) | (pl.col('sepal_width_zscore') < -3)
df_zscore_outliers_removed = df_zscore.filter(is_outlier_z_score == False)

In [53]:
df_zscore.filter(is_outlier_z_score == True)

sepal_length,sepal_width,petal_length,petal_width,species,species_id,sepal_width_zscore
f64,f64,f64,f64,str,i64,f64
5.7,4.4,1.5,0.4,"""setosa""",1,3.104284


In [54]:
df_zscore_outliers_removed.filter(is_outlier_z_score == True)

sepal_length,sepal_width,petal_length,petal_width,species,species_id,sepal_width_zscore
f64,f64,f64,f64,str,i64,f64


In [55]:
df_zscore_outliers_replaced = (
    df_zscore
    .with_columns(
        pl.when(is_outlier_z_score == True)
        .then(pl.col('sepal_width').mean())
        .otherwise(pl.col('sepal_width'))
        .alias('sepal_width')
    )
)

In [56]:
df_zscore_outliers_replaced.filter(is_outlier_z_score == True)

sepal_length,sepal_width,petal_length,petal_width,species,species_id,sepal_width_zscore
f64,f64,f64,f64,str,i64,f64
5.7,3.054,1.5,0.4,"""setosa""",1,3.104284


In [57]:
import polars as pl

df = pl.DataFrame({'value': [1,2,3,4,5]})
df.with_columns(
    pl.when(pl.col('value') < 3)
    .then(pl.lit('low'))
    .otherwise(pl.lit('high'))
    .alias('category')
)

value,category
i64,str
1,"""low"""
2,"""low"""
3,"""high"""
4,"""high"""
5,"""high"""
